# Offset re-generated event timestamps

The events generated by the re-run of the `track.py` script have new timestamps. To synchronize with the other events (e.g. supervisor state) from the original logs, an offset must be applied.

## Setup

In [9]:
import pickle
import json
from datetime import datetime, timedelta, timezone

from timesignal import EventOrchestra, EventSignal

from lib.timestamp import lcm_timestamp_to_seconds, pdt_timestamp_seconds_to_utc_datetime

## Load orchestra and parsed tracks

In [2]:
with open('data/pickle/orchestra.pkl', 'rb') as f:
    orchestra = pickle.load(f)

In [3]:
orchestra_start_dt = orchestra.start

orchestra[orchestra_start_dt]

{'tracks': {'17': {'x_mm': -1660.7912656452074,
   'y_mm': -1660.7912656452074,
   'z_mm': -1660.7912656452074,
   'width_mm': -1660.7912656452074,
   'height_mm': -1660.7912656452074,
   'score': -1660.7912656452074,
   'association_score': -1660.7912656452074,
   'iou': -1660.7912656452074,
   'pixel_error': -1660.7912656452074}}}

In [4]:
with open('data/json/may_24_25_tracks.json') as f:
    new_track_dicts = json.load(f)

## Compute offset

In [5]:
min_timestamp = min(min(event['timestamp'] for event in track['events']) for track in new_track_dicts)
max_timestamp = max(max(event['timestamp'] for event in track['events']) for track in new_track_dicts)

min_dt = datetime.fromtimestamp(min_timestamp)
max_dt = datetime.fromtimestamp(max_timestamp)

duration = max_dt - min_dt

In [7]:
offset = min_dt - orchestra_start_dt

print(offset)

385 days, 1:54:52.037455


## Shift timestamps

In [8]:
# Do not re-run this cell.
offset = min_dt - orchestra_start_dt
offset_seconds = offset.total_seconds()

for track in new_track_dicts:
    for event in track['events']:
        event['timestamp'] = event['timestamp'] - offset_seconds

## Compose sub-orchestra with offset tracks

In [10]:
def get_datetime(ev: dict) -> datetime:
    return pdt_timestamp_seconds_to_utc_datetime(lcm_timestamp_to_seconds(ev['meta']['timestamp']))


def extract_track_event_signal(track, attr, interpolation='nearest') -> EventSignal:
    return EventSignal(((pdt_timestamp_seconds_to_utc_datetime(e['timestamp']), e[attr]) for e in track['events']), interpolation=interpolation)


offset_tracks_orchestra = EventOrchestra({
    str(track['id']): EventOrchestra({
        'x': extract_track_event_signal(track, 'x_mm', interpolation='linear'),
        'y': extract_track_event_signal(track, 'y_mm', interpolation='linear'),
        'z': extract_track_event_signal(track, 'z_mm', interpolation='linear'),
        'width': extract_track_event_signal(track, 'width_mm', interpolation='linear'),
        'height': extract_track_event_signal(track, 'height_mm', interpolation='linear'),
        'score': extract_track_event_signal(track, 'score'),
    })
    for track in new_track_dicts
})

Extracting track signals...done


## Add offset tracks sub-orchestra to orchestra with `all_tracks` key

In [11]:
# Add new orchestra as sub-orchestra
orchestra._elements['all_tracks'] = offset_tracks_orchestra

In [12]:
# Verify duration is sane
print(orchestra.end - orchestra.start)

1 day, 4:48:18.934981


## Overwrite orchestra

In [14]:
# Overwrite old orchestra
with open('data/pickle/orchestra.pkl', 'wb') as f:
    pickle.dump(orchestra, f)